# Imports

In [14]:
import pandas as pd
from datetime import datetime
from dlt.sources.helpers import requests
from bs4 import BeautifulSoup
from hashlib import md5
import re

# Teste scrappers

## Chaves na mão

### Variáveis iniciais

In [36]:
base_url: str = "https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o="
rent_html_class: str = "olx-ad-card__content olx-ad-card__content--horizontal"
rent_html_element: str = "div"
page_number = 1

### Funções

In [37]:
# Fazer função de filtrar lista de palavras
def filter_words(words_list, desired_c=None, not_desired_c=None):
    # Retornar lista com as palavras contenham todos os desired_c e nenhum dos not_desired_c
    if desired_c and not_desired_c:
        return [word for word in words_list if any(char in word for char in desired_c) and all(char not in word for char in not_desired_c)]
    
    # Retornar lista com as palavras contenham todos os desired_c
    elif desired_c:
        return [word for word in words_list if any(char in word for char in desired_c)]

    # Retornar lista com as palavras não contenham not_desired_c
    elif not_desired_c:
        return [word for word in words_list if all(char not in word for char in not_desired_c)]

# Fazer função de pegar o possível preço do imóvel em uma lista
def get_rent_price(possible_prices: list, max_rent: float = float("inf")) -> float:
    # Fazer lista vazia para guardar os valores que realmente podem ser o preço
    new_possible_prices = []

    # Iterar as possíveis strings que contem o preço
    for price in possible_prices:
        # Se a string iterada tiver qualquer dígito numérico
        if any(l.isdigit() for l in price):
            # Mantenha apenas os numéricos ou ".", "," na string
            price = "".join([l for l in price if l.isdigit() or l in (".", ",")])

            # Em casos de preço com virgula, pegue a primeira parte do preço apenas
            price = price.split(",")[0]

            # Remova . se eles forem começo ou final de string
            price = price.strip(".")

            # Converta . em _
            price = price.replace(".", "_")
            
            # Tente
            try:
                # Conversão para floar
                price = float(price)

                # Se o preço for maior que 100_000, divida-o por esse valor
                if price > 99_999:
                    price = price / 100_000

                # Guarde ela na lista 
                new_possible_prices.append(price)

            # Em caso de erro de tipo na conversão, pule essa string
            except ValueError:
                print(f"String -> {price} não é um preço de imóvel!")

    # Filtrar alugueis abaixo de max_rent
    new_possible_prices = [price for price in new_possible_prices if price < max_rent]

    # Tente
    try:
        # Retornae o maior campo da lista
        return max(new_possible_prices)
    
    # Em caso de lista vazia
    except ValueError:
        # Retorne nuloe
        return None

# Fazer função de pegar o tamanho do imóvel
def get_rent_size(possible_sizes: list,  remove_from_size_chars: tuple, max_size: int = float("inf")) -> float:
    # Fazer lista vazia para guardar os valores que realmente podem ser o tamanho do imóvel
    new_possible_sizes = []

    # Iterar as possíveis strings que contem o tamanho
    for size in possible_sizes:
        # Se a string iterada tiver qualquer dígito numérico
        if any(l.isdigit() for l in size):
            # Retira todos os remove_from_size_chars da string
            for char in remove_from_size_chars:
                size = size.replace(char, "")

            # Deixe somente numeros e "," no size
            size = "".join([l for l in size if l.isdigit() or l == ","])

            # Divida o tamanho na "," (se tiver) e pegue o primeiro campo
            size = size.split(",")[0]

            # Tente
            try:
                # Conversão para inteiro
                size = int(size)
            
                # Guarde ela na lista
                new_possible_sizes.append(size)

            # Em caso de erro de tipo na conversão, pule essa string
            except ValueError:
                print(f"String -> {size} não é um tamanho de imóvel!")

    # Filtrar tamanhnos abaixo de max_size
    new_possible_sizes = [size for size in new_possible_sizes if size < max_size]

    # Tente retornar o primeiro indice
    try:
        # Retorne o primeiro campo da lista
        return new_possible_sizes[0]
    
    # Se não houver itens o suficiente
    except IndexError:
        # Retorne Nulo
        return None
    
# Fazer função de pegar o endereço do imóvel
def get_rent_adress(rent_splited_words: list) -> str:
    # Importar objetos   
    from rent_warehouse_mania_pipeline_objects import street_synonyms, city_names
    
    # Faça uma lista vazia para guardar os possíveis indexes
    possible_rua_index = []
    possible_city_index = []

    # Iterar todas as palavras buscando os possíveis indices da rua
    for iword, word in enumerate(rent_splited_words):
        # Se a palavra estiver na lista de sinonimos para rua, guarde esse index
        if any(i in word.lower() for i in street_synonyms):
            possible_rua_index.append(iword)

        # Se a palavra estiver na lista de cidades, guarde esse index
        if any(i in word.lower() for i in city_names):
            possible_city_index.append(iword)

    # Se ambas as listas tiverem index
    if len(possible_rua_index) != 0 and len(possible_city_index) != 0:
        # Retornar o join de todas as palavras na string, escolhendo o min(possible_rua_index) até o max(possible_city_index)
        return " ".join(rent_splited_words[min(possible_rua_index):max(possible_city_index)])
    
    # Do contrário
    else:
        # Retorne um join geral
        return " ".join(rent_splited_words)


### Testes

### Scrapper

In [38]:
di =  {
    "id": [],
    "datahora": [],
    "preco": [],
    "tamanho": [],
    "endereco":[],
}

while True:
    # Definir url pagina atual
    url = base_url + f"{page_number}"

    # Mostra página atual iterada
    print(f"URL Base -> {base_url};\nPágina iterada atualmente -> {page_number}")

    # Tentar pegar a response
    try:
        response = requests.get(url, allow_redirects=False)

    # Em caso de erro, pare a função
    except:
        # Mostre a url atual
        print(f"Data scrapper tentou acessar a página [{page_number}] da URL base. \nNão obteve HTML na resposta")
        
        # Pare a função
        break 

    # Se o status vier 200, prossiga
    if response.status_code == 200:
        # Pegar sopa de letras com o BeautifulSoup
        soup = BeautifulSoup(response.content, "html.parser")

        # Pegar todas as divs com a classe rent_html_class
        imoveis = [imovel.text for imovel in soup.find_all(rent_html_element, class_=rent_html_class)]

        # Iterar todos os imóveis
        for imovel in imoveis:
            # Splitar palavras do card do imovel
            imovel_words = imovel.split()

            # Pegar campo de preço do imovel
            preco = get_rent_price(filter_words(imovel_words, desired_c=(".", "$"), not_desired_c="²"), max_rent=50_000)

            # Pegar campo de tamanho
            tamanho = get_rent_size(filter_words(imovel_words, desired_c=("²"), not_desired_c="$"), remove_from_size_chars=("²"), max_size=5_000)

            # Pegar campo de endereço
            endereco = get_rent_adress(imovel_words)

            # Gerar id com hash md5
            rent_id = md5(endereco.encode("utf-8")).hexdigest()

            # Retornar o dicionários com os dados do imóvel
            di["id"].append(rent_id)
            di["datahora"].append(datetime.strptime(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S"))
            di["preco"].append(preco)
            di["tamanho"].append(tamanho)
            di["endereco"].append(endereco)

        # Incrementar pagina para próximo yield
        page_number += 1

    # Se o status não for 200
    else:
        # Mostre a url atual
        print(f"Data scrapper tentou acessar a página [{page_number}] da URL base. \nNão obteve HTML na resposta")
        
        # Pare a função
        break 

URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada atualmente -> 1
URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada atualmente -> 2
URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada atualmente -> 3
URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada atualmente -> 4
URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada atualmente -> 5
URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada atualmente -> 6
URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada atualmente -> 7
URL Base -> https://www.olx.com.br/imoveis/aluguel/estado-pr/regiao-de-curitiba-e-paranagua?o=;
Página iterada 

In [39]:
pd.DataFrame(di)

,id,datahora,preco,tamanho,endereco
0,db4e5f4fede940bc43be83673ff8fd42,2023-12-13 14:24:35,4600.0,87.0,APARTAMENTO com 3 dormitórios para alugar com ...
1,a7220c62960fb4f2da1a536544aa5135,2023-12-13 14:24:35,1800.0,325.0,Sobrado 04 dormitórios sendo 02 suítes Matinho...
2,f4a279b8b8f484a92a366e47de32ebe2,2023-12-13 14:24:35,2600.0,150.0,Sobrado Sobrado para locação em Curitiba - PRR...
3,f73f59dc91ea62ecc79bd494678251df,2023-12-13 14:24:35,2100.0,19.0,Studio para aluguel no RebouçasR$ 2.100 1 19m²...
4,084c2c26ce18db9cbd334840065bcfd5,2023-12-13 14:24:35,4600.0,83.0,Apartamento Duplex com 1 quarto e 2 vagas no B...
...,...,...,...,...,...
650,a4786c342d50a06c314972dc8068c588,2023-12-13 14:25:45,6100.0,61.0,Apartamento com 2 quartos para alugar por R$ 2...
651,bf24730631ddee58788846f988dfb31f,2023-12-13 14:25:45,2660.0,69.0,"Apartamento para Aluguel - Cidade Industrial, ..."
652,e58aeeac817c5a87b2254f25f77d17a5,2023-12-13 14:25:45,1220.0,38.0,"Apartamento para Aluguel - Tatuquara, 2 Quarto..."
653,9e6ad13ca16bb5dc7b9c80b519404db4,2023-12-13 14:25:45,2056.0,120.0,"Casa para Aluguel - Uberaba, 3 Quartos, 120 m2..."


In [34]:
dipd

,id,datahora,preco,tamanho,endereco
0,cf5300a245853453027ebd9a3e64df9c,2023-12-13 14:12:47,1150.0,35.0,Kitinete mobiliada para mulher R$ 1.150 1 35m²...
1,203c21b3c3505c4fea8724ad4c850231,2023-12-13 14:12:47,NaN,NaN,Casa natal réveillon GuaratubaR$ 500 3 1 2Dire...
2,5302947e3868510629bf0088ede3ca68,2023-12-13 14:12:47,6500.0,144.0,Apartamento mobiliado EcovilleR$ 6.500 3 144m²...
3,fd13d41cbbcd275198ac39a6e0236041,2023-12-13 14:12:47,1200.0,65.0,Sobrado 02 quartos para locação mensal R$ 1.20...
4,fd13d41cbbcd275198ac39a6e0236041,2023-12-13 14:12:47,1200.0,65.0,Sobrado 02 quartos para locação mensal R$ 1.20...
5,cf073830d7248840b5eafe2969514654,2023-12-13 14:12:47,1000.0,47.0,CONJ. RESID. BARIGUI - Ap com 02 dormitórios p...
6,d41d8cd98f00b204e9800998ecf8427e,2023-12-13 14:12:47,2700.0,127.0,
7,bf2c6159ff1331fdbdf81842ee22d026,2023-12-13 14:12:47,2950.0,160.0,Casa residencial com 2 dormitórios para alugar...
8,6b55055bf23b5fe13e1c2be505c85168,2023-12-13 14:12:48,1280.0,40.0,Apartamento de 01 quarto no Centro em Curitiba...
9,4ca4ba2fe913d9fa4c2df95ca01cc2ed,2023-12-13 14:12:48,1500.0,65.0,Aluga apartamento Fazenda Rio GrandeR$ 1.500 2...
